In [1]:
import timeit
from collections import deque
from array import array
import numpy as np
import gc

# -----------------------------
# Core operation-only benchmarks
# -----------------------------
def bench_list_ops(n):
    lst = []            # creation: trivial, allowed
    for i in range(n):
        lst.insert(0, i)   # O(n)
    for i in range(n):
        lst.pop(0)         # O(n)

def bench_deque_ops(n):
    dq = deque()           # creation: trivial, allowed
    for i in range(n):
        dq.appendleft(i)   # O(1)
    for i in range(n):
        dq.popleft()       # O(1)

def bench_array_ops(n):
    arr = array('i')       # creation: trivial, allowed
    for i in range(n):
        arr.insert(0, i)   # O(n)
    for i in range(n):
        arr.pop(0)         # O(n)

# NumPy: pre-create outside timing to avoid counting creation.
# We'll simulate "insert at front" by shifting right in-place using a scratch buffer.
def make_numpy_worker(n):
    arr = np.zeros(n, dtype=np.int32)      # not timed
    buf = np.empty_like(arr)               # not timed

    def run():
        # insert at front by shifting right
        for i in range(n):
            # shift right by 1 using the buffer (no new allocations each iteration)
            buf[:] = arr
            arr[1:] = buf[:-1]
            arr[0] = i
        # "pop from front" by shifting left
        for _ in range(n):
            buf[:] = arr
            arr[:-1] = buf[1:]
            # arr[-1] becomes "vacated"; leave as-is (doesn't affect complexity)
    return run

# -----------------------------
# Timing harness (ops only)
# -----------------------------
def time_func(fn, number=1, repeat=5, warmup=1):
    # Warmup (JITs/CPU caches, etc.)
    for _ in range(warmup):
        fn()
    # Disable GC during measurement for less noise
    old_gc = gc.isenabled()
    gc.disable()
    try:
        times = timeit.repeat(fn, number=number, repeat=repeat)
    finally:
        if old_gc:
            gc.enable()
    return min(times) / number  # best-of

if __name__ == "__main__":
    n = 10_000

    # Prepare NumPy worker outside timing (so creation isn't counted)
    numpy_worker = make_numpy_worker(n)

    # Time each benchmark
    t_list  = time_func(lambda: bench_list_ops(n))
    t_deque = time_func(lambda: bench_deque_ops(n))
    t_array = time_func(lambda: bench_array_ops(n))
    t_numpy = time_func(numpy_worker)

    print({
        "list_insert_pop_front": t_list,
        "deque_appendleft_popleft": t_deque,
        "array_insert_pop_front": t_array,
        "numpy_shift_simulated": t_numpy,
    })


{'list_insert_pop_front': 0.046349625001312234, 'deque_appendleft_popleft': 0.0007172080004238524, 'array_insert_pop_front': 0.008418458004598506, 'numpy_shift_simulated': 0.044806582998717204}


In [3]:
import timeit
import gc

# ---------- Timing harness (ops-only) ----------
def time_func(fn, number=1, repeat=5, warmup=1):
    for _ in range(warmup):
        fn()
    old_gc = gc.isenabled()
    gc.disable()
    try:
        times = timeit.repeat(fn, number=number, repeat=repeat)
    finally:
        if old_gc:
            gc.enable()
    return min(times) / number

# ---------- Workers that EXCLUDE creation ----------
def make_read_worker(n):
    lst = list(range(n))               # precreate once (not timed)
    def run():
        for i in range(n):
            _ = lst[i]
    return run

def make_append_worker(n):
    lst = []                           # creation excluded
    def run():
        lst.clear()
        for i in range(n):
            lst.append(i)
    return run

def make_insert_front_worker(n):
    lst = []
    def run():
        lst.clear()
        for i in range(n):
            lst.insert(0, i)
    return run

def make_pop_worker(n, total_calls):
    pools = [list(range(n)) for _ in range(total_calls)]  # not timed
    idx = {"i": 0}
    def run():
        lst = pools[idx["i"]]
        idx["i"] += 1
        for _ in range(n):
            lst.pop()
    return run

def make_remove_worker(n, total_calls):
    pools = [list(range(n)) for _ in range(total_calls)]  # not timed
    idx = {"i": 0}
    def run():
        lst = pools[idx["i"]]
        idx["i"] += 1
        for i in range(n):
            lst.remove(i)
    return run

def make_index_worker(n, total_calls):
    pools = [list(range(n)) for _ in range(total_calls)]  # not timed
    idx = {"i": 0}
    def run():
        lst = pools[idx["i"]]
        idx["i"] += 1
        for i in range(n):
            _ = lst.index(i)
    return run

# ---------- Run benchmarks ----------
if __name__ == "__main__":
    n = 10_000
    repeats = 5
    number  = 1
    warmup  = 1

    total_calls = warmup + repeats * number

    t_read   = time_func(make_read_worker(n), number=number, repeat=repeats, warmup=warmup)
    t_insert = time_func(make_insert_front_worker(n), number=number, repeat=repeats, warmup=warmup)
    t_append = time_func(make_append_worker(n), number=number, repeat=repeats, warmup=warmup)
    t_pop    = time_func(make_pop_worker(n, total_calls), number=number, repeat=repeats, warmup=warmup)
    t_remove = time_func(make_remove_worker(n, total_calls), number=number, repeat=repeats, warmup=warmup)
    t_index  = time_func(make_index_worker(n, total_calls), number=number, repeat=repeats, warmup=warmup)

    print({
        "list_read_indexing": t_read,
        "list_insert_front": t_insert,
        "list_append_back": t_append,
        "list_pop_back": t_pop,
        "list_remove_value": t_remove,
        "list_index_search": t_index,
    })


{'list_read_indexing': 0.0003162909997627139, 'list_insert_front': 0.03871883300598711, 'list_append_back': 0.000333791998855304, 'list_pop_back': 0.00036987499333918095, 'list_remove_value': 0.007577958000183571, 'list_index_search': 0.3494160420013941}


In [7]:
# Benchmark functions
def benchmark_set_add(n):
    s = set()
    for i in range(n):
        s.add(i)

def benchmark_set_remove(n):
    s = {i for i in range(n)}
    for i in range(n):
        s.remove(i)

def benchmark_set_membership(n):
    s = {i for i in range(n)}
    for i in range(n):
        x = i in s  # Membership test

def benchmark_set_discard(n):
    s = {i for i in range(n)}
    for i in range(n):
        s.discard(i)

def benchmark_set_clear(n):
    s = {i for i in range(n)}
    s.clear()

def benchmark_set_union(n):
    s1 = {i for i in range(n)}
    s2 = {i for i in range(n, 2*n)}
    s = s1.union(s2)

def benchmark_set_intersection(n):
    s1 = {i for i in range(n)}
    s2 = {i for i in range(int(n/2), n + int(n/2))}
    s = s1.intersection(s2)

def benchmark_set_difference(n):
    s1 = {i for i in range(n)}
    s2 = {i for i in range(int(n/2), n + int(n/2))}
    s = s1.difference(s2)

# Benchmarking
n = 10000
time_set_add = timeit.timeit(lambda: benchmark_set_add(n), number=100)
time_set_remove = timeit.timeit(lambda: benchmark_set_remove(n), number=100)
time_set_membership = timeit.timeit(lambda: benchmark_set_membership(n), number=100)
time_set_discard = timeit.timeit(lambda: benchmark_set_discard(n), number=100)
time_set_clear = timeit.timeit(lambda: benchmark_set_clear(n), number=100)
time_set_union = timeit.timeit(lambda: benchmark_set_union(n), number=100)
time_set_intersection = timeit.timeit(lambda: benchmark_set_intersection(n), number=100)
time_set_difference = timeit.timeit(lambda: benchmark_set_difference(n), number=100)

# Results
print('Add:', time_set_add)
print('Remove:', time_set_remove)
print('Membership:', time_set_membership)
print('Discard:', time_set_discard)
print('Clear:', time_set_clear)
print('Union:', time_set_union)
print('Intersection:', time_set_intersection)
print('Difference:', time_set_difference)

Add: 0.016842254990478978
Remove: 0.03229504299815744
Membership: 0.03208695098874159
Discard: 0.031928241020068526
Clear: 0.012840510986279696
Union: 0.07677349197911099
Intersection: 0.03397759099607356
Difference: 0.03358016998390667
